In [ ]:
import os
import cv2
import numpy as np
import random
from scipy.ndimage import gaussian_filter

def add_non_homogeneous_haze(image, alpha=2, scale=20.0, haze_color=(0, 0, 0)):
    print(haze_color)
    height, width = image.shape[:2]

    center_x, center_y = width // 2, height // 2
    y, x = np.indices((height, width))

    distance_to_center = np.sqrt((x - center_x) ** 2 + (y - center_y) ** 2)
    max_distance = np.sqrt(center_x ** 2 + center_y ** 2)

    # Perlin noise
    haze_mask = np.random.normal(0, 1, (height, width))
    haze_mask = gaussian_filter(haze_mask, sigma=scale)
    haze_mask = (haze_mask - np.min(haze_mask)) / (np.max(haze_mask) - np.min(haze_mask))

    haze_intensity = alpha * haze_mask * (1 - distance_to_center / max_distance)

    # Convert the haze color to a numpy array
    haze_color = np.array(haze_color)

    # Adjust the color channels of the hazy image
    hazy_image = np.clip(image + (haze_intensity[:, :, np.newaxis] * (haze_color - image)), 0, 255).astype(np.uint8)

    return hazy_image

def add_haze(image, alpha=1, direction_func=None, haze_color=(0, 0, 0)):
    height, width = image.shape[:2]
    hazy_image = np.zeros_like(image)
    print(haze_color)
    if direction_func is None:
        direction_func_options = {
            "top_to_bottom": lambda i, j: alpha * (1 - i / height),
            "bottom_to_top": lambda i, j: alpha * (i / height),
            "left_to_right": lambda i, j: alpha * (1 - j / width),
            "right_to_left": lambda i, j: alpha * (j / width),
            "center_to_corner": lambda i, j: alpha * (1 - np.sqrt((j - width // 2) ** 2 + (i - height // 2) ** 2) / np.sqrt((width // 2) ** 2 + (height // 2) ** 2)),
            "corner_to_center": lambda i, j: alpha * (np.sqrt((j - width // 2) ** 2 + (i - height // 2) ** 2) / np.sqrt((width // 2) ** 2 + (height // 2) ** 2)),
        }
        direction_func = direction_func_options[np.random.choice(list(direction_func_options.keys()))]

    i, j = np.indices((height, width))
    haze_intensity = direction_func(i, j)

    # Convert the haze color to a numpy array
    haze_color = np.array(haze_color)

    # Adjust the color channels of the hazy image
    hazy_image = np.clip(image + (haze_intensity[:, :, np.newaxis] * (haze_color - image)), 0, 255).astype(np.uint8)

    return hazy_image

input_folder = './Dataset/TRAIN_CUSTOM'
output_folder_random = './Dataset/NEW_MASK'
os.makedirs(output_folder_random, exist_ok=True)

if not os.path.exists(output_folder_random):
    os.makedirs(output_folder_random)
    
# white = 0
# black = 0
for filename in os.listdir(input_folder):
    if filename.endswith(".jpg"):
        image_path = os.path.join(input_folder, filename)
        original_image = cv2.imread(image_path)
        random_bit = random.choice([0, 1]) 
        # random_i  = random.choice([0, 1])
        
        # if random_i:
        #     haze_color = (0, 0, 0)
        #     black += 1
        # else:
        #     haze_color = (255, 255, 255)
        #     white += 1
        haze_color = (0, 0, 0)
        if random_bit:
            hazy_image = add_haze(original_image, haze_color=haze_color)
        else:
            hazy_image = add_non_homogeneous_haze(original_image, haze_color=haze_color)

        output_path = os.path.join(output_folder_random, f"hazy_{filename}")
        cv2.imwrite(output_path, hazy_image)
        print(f"{filename} processed with random haze direction and {haze_color} haze color, and saved as {output_path}")
print(f"Black = {black} & White = {white}")
print("Image processing complete.")


In [1]:
import os

folder_path = './Dataset/NEW_MASK'  # Replace with the path to your folder
file_count = 0

# Iterate through the files in the folder
for root, dirs, files in os.walk(folder_path):
    file_count += len(files)

print(f"Total number of files in the folder: {file_count}")

Total number of files in the folder: 39006
